In [2]:
#!/usr/bin/env python3
"""
S-band UPLINK (Ground -> Satellite) link margin vs elevation for Satlab SRS-4.

Reads s_up_info.csv (column "Satlab SRS-4") and computes per-elevation uplink margin:

  P_iso(dBm) = EIRP(dBW) - L_total(dB) + 30
  P_rx(dBm)  = P_iso + G_rx(dBi) + L_rx_interconnect(dB) - rolloff(dB) - other_overall(dB)
  Margin(dB) = P_rx(dBm) - sensitivity(dBm)

Notes
- Interconnect is used *as-is*: negative = loss, positive = gain.
- Includes “Other overall (constant)” as an additional loss in the RX chain.

Outputs:
- Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv
- Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.png
"""

import os
import re
import math
import argparse
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

EARTH_RADIUS_KM = 6371.0
TARGET_COLUMN = "Satlab SRS-4"  # column in s_up_info.csv

# ---------- helpers ----------
def to_float(x):
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return None
    s = str(x).strip().replace("%", "").replace(",", "")
    if not s or s.lower() == "nan":
        return None
    try:
        return float(s)
    except Exception:
        return None

def resolve_csv_path(cli_csv: str | None):
    if cli_csv and cli_csv.lower().endswith(".csv") and Path(cli_csv).is_file():
        return cli_csv
    for p in ["./s_up_info.csv", os.path.expanduser("~/Downloads/s_up_info.csv"), "/mnt/data/s_up_info.csv"]:
        if Path(p).is_file():
            return p
    raise FileNotFoundError("CSV not found. Pass --csv /path/to/s_up_info.csv or place s_up_info.csv in the current folder.")

def get_by_python_or_parameter(df: pd.DataFrame, key: str | None, param_regex: str | None, default=None):
    """Try python_variable exact match first; else case-insensitive Parameter regex."""
    if key:
        s = df.loc[df["python_variable"].astype(str) == key, TARGET_COLUMN]
        if not s.empty:
            v = to_float(s.iloc[0])
            if v is not None:
                return v
    if param_regex:
        pat = re.compile(param_regex, re.I)
        s = df.loc[df["Parameter"].astype(str).str.contains(pat, na=False), TARGET_COLUMN]
        if not s.empty:
            v = to_float(s.iloc[0])
            if v is not None:
                return v
    return default

def get_units_by_python_or_parameter(df: pd.DataFrame, key: str | None, param_regex: str | None):
    if key:
        s = df.loc[df["python_variable"].astype(str) == key, "Units"]
        if not s.empty:
            return str(s.iloc[0]).strip()
    if param_regex:
        pat = re.compile(param_regex, re.I)
        s = df.loc[df["Parameter"].astype(str).str.contains(pat, na=False), "Units"]
        if not s.empty:
            return str(s.iloc[0]).strip()
    return ""

def slant_range_km(alt_km: float, elev_deg: float) -> float:
    e = math.radians(elev_deg)
    R = EARTH_RADIUS_KM
    H = R + alt_km
    return -R * math.sin(e) + math.sqrt((R * math.sin(e)) ** 2 + H ** 2 - R ** 2)

def fspl_db(range_km: float, freq_ghz: float) -> float:
    # 92.45 + 20log10(R[km]) + 20log10(f[GHz])
    return 92.45 + 20 * math.log10(max(range_km, 1e-9)) + 20 * math.log10(freq_ghz)

# ---------- main compute ----------
def compute_table(csv_path: str,
                  altitudes=(500, 600),
                  elevations=(0, 5, 10, 15, 20)) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # Frequency (GHz), unit-robust
    f_raw = get_by_python_or_parameter(df, "selected_carrier_frequency", r"Selected\s+Carrier\s+Frequency")
    f_units = get_units_by_python_or_parameter(df, "selected_carrier_frequency", r"Selected\s+Carrier\s+Frequency").lower()
    if f_raw is None:
        raise ValueError("Selected carrier frequency not found in s_up_info.csv")
    if "ghz" in f_units:
        f_ghz = f_raw
    elif "mhz" in f_units:
        f_ghz = f_raw / 1e3
    elif "hz" in f_units:
        f_ghz = f_raw / 1e9
    else:
        f_ghz = f_raw  # assume GHz

    # Transmitter (ground): Equivalent/Given EIRP (W -> dBW)
    eirp_w = get_by_python_or_parameter(df, None, r"Equivalent\s+EIRP")
    if eirp_w is None:
        eirp_w = get_by_python_or_parameter(df, None, r"Given\s+EIRP")
    if eirp_w is None:
        raise ValueError("Equivalent/Given EIRP not found in s_up_info.csv")
    eirp_dbw = 10 * math.log10(eirp_w)

    # Receiver (satellite) chain
    g_rx_dbi = get_by_python_or_parameter(df, None, r"^Peak\s+Gain$|Receiver.*Gain|Antenna\s+Gain") or 0.0
    l_rx_interconn_db = get_by_python_or_parameter(df, "interconnect_losses_gains", r"Interconnect\s+losses.*gains") or 0.0
    rolloff_db = get_by_python_or_parameter(df, None, r"Gain\s+reduction\s+at\s+pointing\s+error") or 0.0
    other_overall_db = get_by_python_or_parameter(df, None, r"Other\s+overall", 0.0) or 0.0

    # Propagation losses
    atm  = get_by_python_or_parameter(df, "atmospheric_attenuation", r"Atmospheric\s+attenuation", 0.0) or 0.0
    rain = get_by_python_or_parameter(df, "rain_attenuation", r"Rain\s+attenuation", 0.0) or 0.0
    scint = get_by_python_or_parameter(df, "scintillation_loss", r"Scintillation\s+loss", 0.0) or 0.0
    pol  = get_by_python_or_parameter(df, "polarisation_loss", r"Polarisation\s+loss|Polarization\s+loss", 0.0) or 0.0
    other_prop = get_by_python_or_parameter(df, "other_propagation_losses", r"Other\s+propagation\s+losses", 0.0) or 0.0

    # Receiver sensitivity (dBm)
    sens_dbm = get_by_python_or_parameter(df, None, r"Receiver\s+sensitivity.*MODCOD.*lookup")
    if sens_dbm is None:
        sens_dbm = get_by_python_or_parameter(df, None, r"Receiver\s+sensitivity")
    if sens_dbm is None:
        raise ValueError("Receiver sensitivity (dBm) not found in s_up_info.csv")

    rows = []
    for alt in altitudes:
        for el in elevations:
            sr_km = slant_range_km(alt, el)
            Lfs = fspl_db(sr_km, f_ghz)
            Ltotal = Lfs + atm + rain + scint + pol + other_prop

            # Isotropic power at satellite antenna input
            p_iso_dbm = eirp_dbw - Ltotal + 30.0

            # Apply RX antenna, interconnect (as-is), rolloff, and other overall loss
            p_rx_dbm = p_iso_dbm + g_rx_dbi + l_rx_interconn_db - rolloff_db - other_overall_db

            margin = p_rx_dbm - sens_dbm

            rows.append({
                "orbit_altitude_km": float(alt),
                "elevation_deg": float(el),
                "frequency_ghz": f_ghz,
                "slant_range_km": sr_km,
                "fspl_db": Lfs,
                "propagation_losses_db": atm + rain + scint + pol + other_prop,
                "total_path_loss_db": Ltotal,
                "eirp_dbw": eirp_dbw,
                "rx_gain_dbi": g_rx_dbi,
                "rx_interconnect_db": l_rx_interconn_db,
                "rx_rolloff_db": rolloff_db,
                "rx_other_overall_db": other_overall_db,
                "p_iso_dbm": p_iso_dbm,
                "p_rx_dbm": p_rx_dbm,
                "rx_sensitivity_dbm": sens_dbm,
                "link_margin_db": margin,
            })

    return pd.DataFrame(rows)

def make_plot(df_out: pd.DataFrame, png_path: str):
    plt.figure(figsize=(8, 6))
    for alt in sorted(df_out["orbit_altitude_km"].unique()):
        subset = df_out[df_out["orbit_altitude_km"] == alt].sort_values("elevation_deg")
        plt.plot(subset["elevation_deg"], subset["link_margin_db"], marker="o", label=f"{int(alt)} km")
    plt.title("Uplink Link Margin vs Elevation — Satlab SRS-4 (S-band)")
    plt.xlabel("Elevation (degrees)")
    plt.ylabel("Link Margin (dB)")
    plt.xticks(sorted(df_out["elevation_deg"].unique()))
    plt.grid(True)
    plt.legend()
    plt.savefig(png_path, bbox_inches="tight")
    plt.close()

def main():
    parser = argparse.ArgumentParser(description="Generate S-band Uplink Link Margin vs Elevation (Satlab SRS-4).")
    parser.add_argument("--csv", help="Path to s_up_info.csv")
    parser.add_argument("--out-csv", default="Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv")
    parser.add_argument("--out-png", default="Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.png")
    parser.add_argument("--alts", nargs="*", type=float, default=[500, 600], help="Altitudes in km")
    parser.add_argument("--elevs", nargs="*", type=float, default=[0, 5, 10, 15, 20], help="Elevations in degrees")
    args, _ = parser.parse_known_args()

    csv_path = resolve_csv_path(args.csv)
    df_out = compute_table(csv_path, altitudes=args.alts, elevations=args.elevs)
    df_out.to_csv(args.out_csv, index=False)
    make_plot(df_out, args.out_png)
    print(f"Wrote CSV -> {args.out_csv}")
    print(f"Wrote PNG -> {args.out_png}")

if __name__ == "__main__":
    main()


Wrote CSV -> Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv
Wrote PNG -> Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.png


In [4]:
#!/usr/bin/env python3
"""
Satlab SRS-4 — S-band UPLINK (Ground -> Satellite)
Link margin vs elevation for given altitudes; exports CSV + PNG.

Uses s_up_info.csv (column "Satlab SRS-4"):

P_iso(dBm) = EIRP(dBW) - L_total(dB) + 30
P_rx(dBm)  = P_iso + G_rx(dBi) + L_rx_interconnect(dB) - rolloff(dB) - other_overall(dB)
Margin(dB) = P_rx(dBm) - sensitivity(dBm)

Conventions:
- Interconnect is used AS-IS (negative = loss, positive = gain).
- Includes “Gain reduction at pointing error” and “Other overall (constant)” as losses.

Outputs:
- Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv
- Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.png
"""

import os
import re
import math
import argparse
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

EARTH_RADIUS_KM = 6371.0
TARGET_COLUMN = "Satlab SRS-4"  # column in s_up_info.csv

# ---------------- helpers ----------------
def to_float(x):
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return None
    s = str(x).strip().replace("%", "").replace(",", "")
    if not s or s.lower() == "nan":
        return None
    try:
        return float(s)
    except Exception:
        return None

def resolve_csv_path(cli_csv: str | None):
    if cli_csv and cli_csv.lower().endswith(".csv") and Path(cli_csv).is_file():
        return cli_csv
    for p in ["./s_up_info.csv", os.path.expanduser("~/Downloads/s_up_info.csv"), "/mnt/data/s_up_info.csv"]:
        if Path(p).is_file():
            return p
    raise FileNotFoundError("CSV not found. Pass --csv /path/to/s_up_info.csv or place s_up_info.csv here.")

def get_by_python_or_parameter(df: pd.DataFrame, key: str | None, param_regex: str | None, default=None):
    """Prefer python_variable exact match; else regex on 'Parameter'."""
    if key:
        s = df.loc[df["python_variable"].astype(str) == key, TARGET_COLUMN]
        if not s.empty:
            v = to_float(s.iloc[0])
            if v is not None:
                return v
    if param_regex:
        pat = re.compile(param_regex, re.I)
        s = df.loc[df["Parameter"].astype(str).str.contains(pat, na=False), TARGET_COLUMN]
        if not s.empty:
            v = to_float(s.iloc[0])
            if v is not None:
                return v
    return default

def get_units_by_python_or_parameter(df: pd.DataFrame, key: str | None, param_regex: str | None):
    if key:
        s = df.loc[df["python_variable"].astype(str) == key, "Units"]
        if not s.empty:
            return str(s.iloc[0]).strip()
    if param_regex:
        pat = re.compile(param_regex, re.I)
        s = df.loc[df["Parameter"].astype(str).str.contains(pat, na=False), "Units"]
        if not s.empty:
            return str(s.iloc[0]).strip()
    return ""

def slant_range_km(alt_km: float, elev_deg: float) -> float:
    e = math.radians(elev_deg)
    R = EARTH_RADIUS_KM
    H = R + alt_km
    return -R * math.sin(e) + math.sqrt((R * math.sin(e)) ** 2 + H ** 2 - R ** 2)

def fspl_db(range_km: float, freq_ghz: float) -> float:
    # 92.45 + 20log10(R[km]) + 20log10(f[GHz])
    return 92.45 + 20 * math.log10(max(range_km, 1e-9)) + 20 * math.log10(freq_ghz)

# ---------------- compute ----------------
def compute_table(csv_path: str,
                  altitudes=(500, 600),
                  elevations=(0, 5, 10, 15, 20)) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # Frequency (GHz), robust to units
    f_raw = get_by_python_or_parameter(df, "selected_carrier_frequency", r"Selected\s+Carrier\s+Frequency")
    f_units = get_units_by_python_or_parameter(df, "selected_carrier_frequency", r"Selected\s+Carrier\s+Frequency").lower()
    if f_raw is None:
        raise ValueError("Selected carrier frequency not found in s_up_info.csv")
    if "ghz" in f_units:
        f_ghz = f_raw
    elif "mhz" in f_units:
        f_ghz = f_raw / 1e3
    elif "hz" in f_units:
        f_ghz = f_raw / 1e9
    else:
        f_ghz = f_raw  # assume GHz

    # Ground TX EIRP (W -> dBW)
    eirp_w = get_by_python_or_parameter(df, None, r"Equivalent\s+EIRP")
    if eirp_w is None:
        eirp_w = get_by_python_or_parameter(df, None, r"Given\s+EIRP")
    if eirp_w is None:
        raise ValueError("Equivalent/Given EIRP not found in s_up_info.csv")
    eirp_dbw = 10 * math.log10(eirp_w)

    # Satellite RX chain
    g_rx_dbi = get_by_python_or_parameter(df, None, r"^Peak\s+Gain$|Receiver.*Gain|Antenna\s+Gain") or 0.0
    l_rx_interconn_db = get_by_python_or_parameter(df, "interconnect_losses_gains", r"Interconnect\s+losses.*gains", 0.0) or 0.0
    rolloff_db = get_by_python_or_parameter(df, None, r"Gain\s+reduction\s+at\s+pointing\s+error", 0.0) or 0.0
    other_overall_db = get_by_python_or_parameter(df, None, r"Other\s+overall", 0.0) or 0.0

    # Propagation losses
    atm   = get_by_python_or_parameter(df, "atmospheric_attenuation", r"Atmospheric\s+attenuation", 0.0) or 0.0
    rain  = get_by_python_or_parameter(df, "rain_attenuation", r"Rain\s+attenuation", 0.0) or 0.0
    scint = get_by_python_or_parameter(df, "scintillation_loss", r"Scintillation\s+loss", 0.0) or 0.0
    pol   = get_by_python_or_parameter(df, "polarisation_loss", r"Polarisation\s+loss|Polarization\s+loss", 0.0) or 0.0
    other_prop = get_by_python_or_parameter(df, "other_propagation_losses", r"Other\s+propagation\s+losses", 0.0) or 0.0

    # Receiver sensitivity (dBm)
    sens_dbm = get_by_python_or_parameter(df, None, r"Receiver\s+sensitivity.*MODCOD.*lookup")
    if sens_dbm is None:
        sens_dbm = get_by_python_or_parameter(df, None, r"Receiver\s+sensitivity")
    if sens_dbm is None:
        raise ValueError("Receiver sensitivity (dBm) not found in s_up_info.csv")

    rows = []
    for alt in altitudes:
        for el in elevations:
            sr_km = slant_range_km(alt, el)
            Lfs = fspl_db(sr_km, f_ghz)
            Ltotal = Lfs + atm + rain + scint + pol + other_prop

            # Isotropic received power at the satellite antenna flange
            p_iso_dbm = eirp_dbw - Ltotal + 30.0
            # Apply RX chain: interconnect as-is; subtract roll-off & other overall
            p_rx_dbm = p_iso_dbm + g_rx_dbi + l_rx_interconn_db - rolloff_db - other_overall_db
            margin = p_rx_dbm - sens_dbm

            rows.append({
                "orbit_altitude_km": float(alt),
                "elevation_deg": float(el),
                "frequency_ghz": f_ghz,
                "slant_range_km": sr_km,
                "fspl_db": Lfs,
                "propagation_losses_db": atm + rain + scint + pol + other_prop,
                "total_path_loss_db": Ltotal,
                "eirp_dbw": eirp_dbw,
                "rx_gain_dbi": g_rx_dbi,
                "rx_interconnect_db": l_rx_interconn_db,
                "rx_rolloff_db": rolloff_db,
                "rx_other_overall_db": other_overall_db,
                "p_iso_dbm": p_iso_dbm,
                "p_rx_dbm": p_rx_dbm,
                "rx_sensitivity_dbm": sens_dbm,
                "link_margin_db": margin,
            })

    return pd.DataFrame(rows)

def make_plot(df_out: pd.DataFrame, png_path: str):
    plt.figure(figsize=(8, 6))
    for alt in sorted(df_out["orbit_altitude_km"].unique()):
        subset = df_out[df_out["orbit_altitude_km"] == alt].sort_values("elevation_deg")
        plt.plot(subset["elevation_deg"], subset["link_margin_db"], marker="o", label=f"{int(alt)} km")
    plt.title("Uplink Link Margin vs Elevation — Satlab SRS-4 (S-band)")
    plt.xlabel("Elevation (degrees)")
    plt.ylabel("Link Margin (dB)")
    plt.xticks(sorted(df_out["elevation_deg"].unique()))
    plt.grid(True)
    plt.legend()
    plt.savefig(png_path, bbox_inches="tight")
    plt.close()

def main():
    ap = argparse.ArgumentParser(description="Satlab SRS-4 — S-band UPLINK link margin vs elevation.")
    ap.add_argument("--csv", help="Path to s_up_info.csv")
    ap.add_argument("--alts", nargs="*", type=float, default=[500, 600], help="Altitudes in km")
    ap.add_argument("--elevs", nargs="*", type=float, default=[0, 5, 10, 15, 20], help="Elevations in degrees")
    ap.add_argument("--out-csv", default="Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv", help="Output CSV")
    ap.add_argument("--out-png", default="Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.png", help="Output plot")
    args, _ = ap.parse_known_args()

    csv_path = resolve_csv_path(args.csv)
    df_out = compute_table(csv_path, altitudes=args.alts, elevations=args.elevs)
    df_out.to_csv(args.out_csv, index=False)
    make_plot(df_out, args.out_png)
    print(f"Wrote CSV -> {args.out_csv}")
    print(f"Wrote PNG -> {args.out_png}")

if __name__ == "__main__":
    main()


Wrote CSV -> Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv
Wrote PNG -> Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.png


In [5]:
#!/usr/bin/env python3
"""
Notebook/CLI friendly UPLINK pass data budget generator — Satlab SRS-4 (S-band).
Outputs in MB (Megabytes, decimal; 1 MB = 1e6 bytes).

- Auto mode (no args, great for notebooks):
  * looks for Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv and s_up_info.csv
    in ./ or /mnt/data
  * finds all altitudes in the link CSV
  * writes pass_data_budget_uplink_<alt>km.csv for each altitude

- CLI mode:
  --link-csv <Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv>
  --info-csv <s_up_info.csv>
  --rate-bps <override bits/second>   # optional override
  --alt-km <500> --emin <min_elev_deg> --margin-min <dB> --out-csv <output.csv>
"""

import argparse, math, sys, os, re
from pathlib import Path
import pandas as pd

MU_EARTH = 398600.4418  # km^3/s^2
R_EARTH = 6371.0        # km
INFO_COLUMN = "Satlab SRS-4"  # column in s_up_info.csv

def find_file(name: str):
    for p in [Path(name), Path("/mnt/data")/name]:
        if p.is_file():
            return str(p)
    return None

def _to_float(x):
    try:
        return float(str(x).strip().replace("%","").replace(",",""))
    except Exception:
        return None

def get_supported_rate(info_csv: str, override_bps: float | None = None):
    """Fetch uplink user data rate (bps) from s_up_info.csv, or use override."""
    if override_bps is not None:
        return float(override_bps), "override b/s"

    df = pd.read_csv(info_csv)

    # Try python_variable first (common variants), then Parameter text patterns
    pv_candidates = [
        "supported_user_data_rate",
        "uplink_supported_user_data_rate",
        "uplink_data_rate",
        "tc_rate",
        "telecommand_rate",
        "command_data_rate",
    ]
    param_patterns = [
        r"^Supported\s+user\s+data\s+rate",
        r"Uplink\s+data\s+rate",
        r"Telecommand\s+data\s+rate",
        r"Command\s+data\s+rate",
        r"\bTC\s*rate\b",
    ]

    # 1) python_variable exact match
    s = df[df["python_variable"].astype(str).str.lower().isin(pv_candidates)]
    if not s.empty and INFO_COLUMN in s.columns:
        val = _to_float(s.iloc[0][INFO_COLUMN])
        units = str(s.iloc[0]["Units"]).lower() if "Units" in s.columns else ""
        if val is not None:
            bps = _rate_to_bps(val, units)
            return bps, f"{units or 'b/s (assumed)'}"

    # 2) Parameter regex match
    pat = re.compile("|".join(param_patterns), re.I)
    s2 = df[df["Parameter"].astype(str).str.contains(pat, na=False)]
    if not s2.empty and INFO_COLUMN in s2.columns:
        val = _to_float(s2.iloc[0][INFO_COLUMN])
        units = str(s2.iloc[0]["Units"]).lower() if "Units" in s2.columns else ""
        if val is not None:
            bps = _rate_to_bps(val, units)
            return bps, f"{units or 'b/s (assumed)'}"

    raise ValueError(
        "Could not find an uplink user data rate in s_up_info.csv. "
        "Pass --rate-bps to override."
    )

def _rate_to_bps(val: float, units: str) -> float:
    u = (units or "").lower()
    if "mb" in u:   # Mbps / Mb/s
        return val * 1e6
    if "kb" in u:
        return val * 1e3
    if "b/s" in u or "bps" in u:
        return val
    # default: assume Mbps if unspecified
    return val * 1e6

def central_angle_from_slant(range_km: float, alt_km: float) -> float:
    H = R_EARTH + alt_km
    rho = range_km
    cos_theta = (R_EARTH*R_EARTH + H*H - rho*rho) / (2*R_EARTH*H)
    return math.acos(max(-1.0, min(1.0, cos_theta)))

def mean_motion_rad_s(alt_km: float) -> float:
    a = R_EARTH + alt_km
    return math.sqrt(MU_EARTH / (a*a*a))

def build_budget(link_csv: str, info_csv: str, alt_km: float,
                 emin: float | None, margin_min: float, out_csv: str,
                 rate_bps_override: float | None = None) -> pd.DataFrame:
    df = pd.read_csv(link_csv)
    need = {"orbit_altitude_km","elevation_deg","link_margin_db","slant_range_km"}
    if not need.issubset(df.columns):
        raise ValueError(f"link CSV missing columns: {need - set(df.columns)}")

    dfa = df[df["orbit_altitude_km"].astype(float) == float(alt_km)].copy()
    if dfa.empty:
        raise ValueError(f"No rows for orbit_altitude_km={alt_km}")
    dfa.sort_values("elevation_deg", inplace=True)

    bps, unit_str = get_supported_rate(info_csv, rate_bps_override)
    if emin is None:
        emin = float(dfa["elevation_deg"].min())

    e = dfa["elevation_deg"].to_list()
    m = dfa["link_margin_db"].to_list()
    r = dfa["slant_range_km"].to_list()
    theta = [central_angle_from_slant(x, alt_km) for x in r]
    n = mean_motion_rad_s(alt_km)

    rows = []
    for i in range(len(e)-1):
        e0, e1 = e[i], e[i+1]
        m0, m1 = m[i], m[i+1]
        th0, th1 = theta[i], theta[i+1]

        if max(e0, e1) < emin:
            continue
        e_start, e_end = max(min(e0, e1), emin), max(e0, e1)
        if e_end <= e_start:
            continue

        def lerp(x0, y0, x1, y1, x):
            t = (x - x0) / (x1 - x0)
            return y0 + t * (y1 - y0)

        # ensure ascending for interpolation
        if e1 < e0:
            e0, e1 = e1, e0
            m0, m1 = m1, m0
            th0, th1 = th1, th0

        th_start = lerp(e0, th0, e1, th1, e_start)
        th_end   = lerp(e0, th0, e1, th1, e_end)
        m_start  = lerp(e0, m0,  e1, m1,  e_start)
        m_end    = lerp(e0, m0,  e1, m1,  e_end)

        # time across BOTH sides of pass (rise + set)
        dtheta = max(0.0, th_start - th_end)
        bin_time_s = 2.0 * dtheta / n

        # fraction of this bin that meets the margin threshold
        usable_fraction = 1.0
        if (m_start < margin_min) and (m_end < margin_min):
            usable_fraction = 0.0
        elif (m_start < margin_min) or (m_end < margin_min):
            if m_end != m_start:
                e_cross = e_start + (margin_min - m_start) * (e_end - e_start) / (m_end - m_start)
                usable_fraction = (e_end - e_cross) / (e_end - e_start) if e_start <= e_cross <= e_end else (1.0 if m_end >= margin_min else 0.0)
            else:
                usable_fraction = 0.0 if m_start < margin_min else 1.0

        data_bits = bps * bin_time_s * usable_fraction
        data_MB   = data_bits / (8 * 1e6)   # MB (decimal)
        data_MiB  = data_bits / (8 * 1024**2)

        rows.append({
            "alt_km": float(alt_km),
            "elev_start_deg": float(e_start),
            "elev_end_deg": float(e_end),
            "theta_start_deg": math.degrees(th_start),
            "theta_end_deg": math.degrees(th_end),
            "bin_time_s": bin_time_s,
            "margin_start_db": m_start,
            "margin_end_db": m_end,
            "usable_fraction": usable_fraction,
            "data_bits": data_bits,
            "data_MB": data_MB,      # main output unit
            "data_MiB": data_MiB,    # optional, binary
            "rate_bps": bps,
            "rate_unit": unit_str,
        })

    out = pd.DataFrame(rows)
    out.to_csv(out_csv, index=False)
    print(f"[{int(alt_km)} km] time≥thr: {out['bin_time_s'].sum():.1f}s  data: {out['data_MB'].sum():.2f} MB")
    return out

def main():
    parser = argparse.ArgumentParser(description="Create S-band UPLINK pass data budget for Satlab SRS-4 (MB output).")
    parser.add_argument("--link-csv", required=False, help="Path to Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv")
    parser.add_argument("--info-csv", required=False, help="Path to s_up_info.csv")
    parser.add_argument("--rate-bps", type=float, default=None, help="Override uplink rate in bits/second")
    parser.add_argument("--alt-km", type=float, required=False, help="Orbit altitude to use")
    parser.add_argument("--emin", type=float, default=None, help="Minimum elevation (deg)")
    parser.add_argument("--margin-min", type=float, default=0.0, help="Minimum link margin counted (dB)")
    parser.add_argument("--out-csv", required=False, help="Output CSV path")
    args, _ = parser.parse_known_args()

    # Auto mode (nice for notebooks)
    if len(sys.argv) == 1 or (args.link_csv is None and args.info_csv is None):
        link_csv = find_file("Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv")
        info_csv = find_file("s_up_info.csv")
        if not link_csv or not info_csv:
            raise SystemExit("Auto-mode couldn't find the CSVs. Put them next to the script or pass --link-csv/--info-csv.")
        df = pd.read_csv(link_csv)
        alts = sorted(set(df["orbit_altitude_km"]))
        for alt in alts:
            out_csv = f"pass_data_budget_uplink_{int(alt)}km.csv"
            build_budget(link_csv, info_csv, float(alt), args.emin, args.margin_min, out_csv, args.rate_bps)
        return

    # CLI mode
    if not all([args.link_csv, args.info_csv, args.alt_km, args.out_csv]):
        parser.error("the following arguments are required: --link-csv, --info-csv, --alt-km, --out-csv")

    build_budget(args.link_csv, args.info_csv, args.alt_km, args.emin, args.margin_min, args.out_csv, args.rate_bps)

if __name__ == "__main__":
    main()


[500 km] time≥thr: 396.9s  data: -4370.50 MB
[600 km] time≥thr: 422.5s  data: -3721.30 MB


In [6]:
#!/usr/bin/env python3
"""
Notebook/CLI friendly UPLINK pass data budget generator — Satlab SRS-4 (S-band).
Outputs in MB (Megabytes, decimal; 1 MB = 1e6 bytes).

- Auto mode (no args, great for notebooks):
  * looks for Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv and s_up_info.csv
    in ./ or /mnt/data
  * finds all altitudes in the link CSV
  * writes pass_data_budget_uplink_<alt>km.csv for each altitude

- CLI mode:
  --link-csv <Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv>
  --info-csv <s_up_info.csv>
  --rate-bps <override bits/second>   # optional override
  --alt-km <500> --emin <min_elev_deg> --margin-min <dB> --out-csv <output.csv>
"""

import argparse, math, sys, os, re
from pathlib import Path
import pandas as pd

MU_EARTH = 398600.4418  # km^3/s^2
R_EARTH = 6371.0        # km
INFO_COLUMN = "Satlab SRS-4"  # column in s_up_info.csv
SAFE_DEFAULT_UPLINK_BPS = 9600.0  # conservative fallback if no valid rate is found

def find_file(name: str):
    for p in [Path(name), Path("/mnt/data")/name]:
        if p.is_file():
            return str(p)
    return None

def _to_float(x):
    try:
        return float(str(x).strip().replace("%","").replace(",",""))
    except Exception:
        return None

def _rate_to_bps(val: float, units: str) -> float | None:
    """Convert only if units look like a rate; else return None."""
    u = (units or "").lower()
    if any(tag in u for tag in ("mbps","mb/s","mbit","megabit","megabits","mb")):
        return val * 1e6
    if any(tag in u for tag in ("kbps","kb/s","kbit","kilobit","kilobits","kb")):
        return val * 1e3
    if ("bps" in u) or ("/s" in u) or (u == "b/s"):
        # e.g., bps, bit/s, bits/s
        return val
    # Not a rate unit -> refuse conversion
    return None

def get_supported_rate(info_csv: str, override_bps: float | None = None):
    """
    Fetch uplink user data rate (bps) from s_up_info.csv, or use override.
    Only accepts units that clearly indicate a rate (bps/kbps/Mbps/etc).
    """
    if override_bps is not None:
        if override_bps <= 0:
            raise ValueError("--rate-bps must be > 0")
        return float(override_bps), "override b/s"

    df = pd.read_csv(info_csv)

    # 1) python_variable exact match candidates
    pv_candidates = {
        "supported_user_data_rate",
        "uplink_supported_user_data_rate",
        "uplink_data_rate",
        "tc_rate",
        "telecommand_rate",
        "command_data_rate",
    }
    s = df[df["python_variable"].astype(str).str.lower().isin(pv_candidates)]
    if not s.empty and INFO_COLUMN in s.columns:
        val = _to_float(s.iloc[0][INFO_COLUMN])
        units = str(s.iloc[0]["Units"]).lower() if "Units" in s.columns else ""
        bps = _rate_to_bps(val, units) if val is not None else None
        if bps and bps > 0:
            return bps, (units or "b/s")

    # 2) Parameter regex patterns
    param_patterns = [
        r"^Supported\s+user\s+data\s+rate",
        r"Uplink\s+data\s+rate",
        r"Telecommand\s+data\s+rate",
        r"Command\s+data\s+rate",
        r"\bTC\s*rate\b",
    ]
    pat = re.compile("|".join(param_patterns), re.I)
    s2 = df[df["Parameter"].astype(str).str.contains(pat, na=False)]
    if not s2.empty and INFO_COLUMN in s2.columns:
        val = _to_float(s2.iloc[0][INFO_COLUMN])
        units = str(s2.iloc[0]["Units"]).lower() if "Units" in s2.columns else ""
        bps = _rate_to_bps(val, units) if val is not None else None
        if bps and bps > 0:
            return bps, (units or "b/s")

    # 3) No valid rate found -> safe default, with a loud note
    print(f"WARNING: No valid uplink data rate found in {info_csv}; "
          f"using conservative default {SAFE_DEFAULT_UPLINK_BPS:.0f} bps. "
          f"Override with --rate-bps if needed.")
    return SAFE_DEFAULT_UPLINK_BPS, "b/s (defaulted)"

def central_angle_from_slant(range_km: float, alt_km: float) -> float:
    H = R_EARTH + alt_km
    rho = range_km
    cos_theta = (R_EARTH*R_EARTH + H*H - rho*rho) / (2*R_EARTH*H)
    return math.acos(max(-1.0, min(1.0, cos_theta)))

def mean_motion_rad_s(alt_km: float) -> float:
    a = R_EARTH + alt_km
    return math.sqrt(MU_EARTH / (a*a*a))

def build_budget(link_csv: str, info_csv: str, alt_km: float,
                 emin: float | None, margin_min: float, out_csv: str,
                 rate_bps_override: float | None = None) -> pd.DataFrame:
    df = pd.read_csv(link_csv)
    need = {"orbit_altitude_km","elevation_deg","link_margin_db","slant_range_km"}
    if not need.issubset(df.columns):
        raise ValueError(f"link CSV missing columns: {need - set(df.columns)}")

    dfa = df[df["orbit_altitude_km"].astype(float) == float(alt_km)].copy()
    if dfa.empty:
        raise ValueError(f"No rows for orbit_altitude_km={alt_km}")
    dfa.sort_values("elevation_deg", inplace=True)

    bps, unit_str = get_supported_rate(info_csv, rate_bps_override)
    if bps <= 0:
        raise ValueError("Parsed uplink rate is non-positive; please set --rate-bps to a positive value.")
    if emin is None:
        emin = float(dfa["elevation_deg"].min())

    e = dfa["elevation_deg"].to_list()
    m = dfa["link_margin_db"].to_list()
    r = dfa["slant_range_km"].to_list()
    theta = [central_angle_from_slant(x, alt_km) for x in r]
    n = mean_motion_rad_s(alt_km)

    rows = []
    for i in range(len(e)-1):
        e0, e1 = e[i], e[i+1]
        m0, m1 = m[i], m[i+1]
        th0, th1 = theta[i], theta[i+1]

        if max(e0, e1) < emin:
            continue
        e_start, e_end = max(min(e0, e1), emin), max(e0, e1)
        if e_end <= e_start:
            continue

        def lerp(x0, y0, x1, y1, x):
            t = (x - x0) / (x1 - x0)
            return y0 + t * (y1 - y0)

        # ensure ascending for interpolation
        if e1 < e0:
            e0, e1 = e1, e0
            m0, m1 = m1, m0
            th0, th1 = th1, th0

        th_start = lerp(e0, th0, e1, th1, e_start)
        th_end   = lerp(e0, th0, e1, th1, e_end)
        m_start  = lerp(e0, m0,  e1, m1,  e_start)
        m_end    = lerp(e0, m0,  e1, m1,  e_end)

        # time across BOTH sides of pass (rise + set)
        dtheta = max(0.0, th_start - th_end)
        bin_time_s = 2.0 * dtheta / n

        # fraction of this bin that meets the margin threshold
        usable_fraction = 1.0
        if (m_start < margin_min) and (m_end < margin_min):
            usable_fraction = 0.0
        elif (m_start < margin_min) or (m_end < margin_min):
            if m_end != m_start:
                e_cross = e_start + (margin_min - m_start) * (e_end - e_start) / (m_end - m_start)
                usable_fraction = (e_end - e_cross) / (e_end - e_start) if e_start <= e_cross <= e_end else (1.0 if m_end >= margin_min else 0.0)
            else:
                usable_fraction = 0.0 if m_start < margin_min else 1.0

        data_bits = bps * bin_time_s * usable_fraction
        data_MB   = data_bits / (8 * 1e6)   # MB (decimal)
        data_MiB  = data_bits / (8 * 1024**2)

        rows.append({
            "alt_km": float(alt_km),
            "elev_start_deg": float(e_start),
            "elev_end_deg": float(e_end),
            "theta_start_deg": math.degrees(th_start),
            "theta_end_deg": math.degrees(th_end),
            "bin_time_s": bin_time_s,
            "margin_start_db": m_start,
            "margin_end_db": m_end,
            "usable_fraction": usable_fraction,
            "data_bits": data_bits,
            "data_MB": data_MB,      # main output unit
            "data_MiB": data_MiB,    # optional, binary
            "rate_bps": bps,
            "rate_unit": unit_str,
        })

    out = pd.DataFrame(rows)
    out.to_csv(out_csv, index=False)
    print(f"[{int(alt_km)} km] time≥thr: {out['bin_time_s'].sum():.1f}s  data: {out['data_MB'].sum():.3f} MB  (rate={bps:.0f} bps)")
    return out

def main():
    parser = argparse.ArgumentParser(description="Create S-band UPLINK pass data budget for Satlab SRS-4 (MB output).")
    parser.add_argument("--link-csv", required=False, help="Path to Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv")
    parser.add_argument("--info-csv", required=False, help="Path to s_up_info.csv")
    parser.add_argument("--rate-bps", type=float, default=None, help="Override uplink rate in bits/second")
    parser.add_argument("--alt-km", type=float, required=False, help="Orbit altitude to use")
    parser.add_argument("--emin", type=float, default=None, help="Minimum elevation (deg)")
    parser.add_argument("--margin-min", type=float, default=0.0, help="Minimum link margin counted (dB)")
    parser.add_argument("--out-csv", required=False, help="Output CSV path")
    args, _ = parser.parse_known_args()

    # Auto mode (nice for notebooks)
    if len(sys.argv) == 1 or (args.link_csv is None and args.info_csv is None):
        link_csv = find_file("Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv")
        info_csv = find_file("s_up_info.csv")
        if not link_csv or not info_csv:
            raise SystemExit("Auto-mode couldn't find the CSVs. Put them next to the script or pass --link-csv/--info-csv.")
        df = pd.read_csv(link_csv)
        alts = sorted(set(df["orbit_altitude_km"]))
        for alt in alts:
            out_csv = f"pass_data_budget_uplink_{int(alt)}km.csv"
            build_budget(link_csv, info_csv, float(alt), args.emin, args.margin_min, out_csv, args.rate_bps)
        return

    # CLI mode
    if not all([args.link_csv, args.info_csv, args.alt_km, args.out_csv]):
        parser.error("the following arguments are required: --link-csv, --info-csv, --alt-km, --out-csv")

    build_budget(args.link_csv, args.info_csv, args.alt_km, args.emin, args.margin_min, args.out_csv, args.rate_bps)

if __name__ == "__main__":
    main()


[500 km] time≥thr: 396.9s  data: 0.417 MB  (rate=9600 bps)
[600 km] time≥thr: 422.5s  data: 0.355 MB  (rate=9600 bps)


In [7]:
#!/usr/bin/env python3
"""
Satlab SRS-4 — S-band UPLINK pass data budget (MB, decimal).

Assumptions:
- Uplink rates are telecommand-class (e.g., 9.6–64 kbps typical; up to ~100–256 kbps if margins allow).
- We integrate only time bins whose margin >= --margin-min (default 0 dB).
- Output is MB (decimal; 1 MB = 1e6 bytes).

Auto mode (no args):
  - Finds: Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv and s_up_info.csv in . or /mnt/data
  - Uses default rate set: [9600, 19200, 64000, 100000, 256000] bps
  - For each altitude in the link CSV, writes:
      pass_data_budget_uplink_<alt>km_<rateKbps>kbps.csv
  - Also writes summary CSV + PNG:
      uplink_pass_capacity_summary.csv / .png

CLI mode:
  --link-csv  <uplink_link_curve.csv>
  --info-csv  <s_up_info.csv>                 # only needed if --use-info-rate
  --rate-bps  <bps> (repeatable)              # override/add multiple rates
  --use-info-rate                             # include single rate parsed from s_up_info.csv
  --alt-km <500> --emin <deg> --margin-min <dB>
  --outdir <dir>

Example:
  python uplink_pass_budget.py --rate-bps 9600 --rate-bps 64000 --margin-min 0
"""

import argparse, math, sys, os, re
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

MU_EARTH = 398600.4418  # km^3/s^2
R_EARTH  = 6371.0       # km
INFO_COLUMN = "Satlab SRS-4"

DEFAULT_RATES_BPS = [9600, 19200, 64000, 100000, 256000]  # realistic TC set

# ---------------- file helpers ----------------
def find_file(name: str):
    for p in [Path(name), Path("/mnt/data")/name]:
        if p.is_file():
            return str(p)
    return None

# ---------------- rate helpers ----------------
def _to_float(x):
    try:
        return float(str(x).strip().replace("%","").replace(",",""))
    except Exception:
        return None

def _rate_to_bps(val: float, units: str) -> float | None:
    """Convert only if units look like a rate; else return None."""
    u = (units or "").lower()
    if any(tag in u for tag in ("mbps","mb/s","mbit","megabit","megabits","mb")):
        return val * 1e6
    if any(tag in u for tag in ("kbps","kb/s","kbit","kilobit","kilobits","kb")):
        return val * 1e3
    if ("bps" in u) or ("/s" in u) or (u == "b/s"):
        return val
    return None

def get_info_rate_bps(info_csv: str) -> float | None:
    """Try to pull ONE uplink rate from s_up_info.csv, else None."""
    if not info_csv:
        return None
    df = pd.read_csv(info_csv)
    # python_variable candidates
    pv_candidates = {
        "supported_user_data_rate",
        "uplink_supported_user_data_rate",
        "uplink_data_rate",
        "tc_rate",
        "telecommand_rate",
        "command_data_rate",
    }
    s = df[df["python_variable"].astype(str).str.lower().isin(pv_candidates)]
    if not s.empty and INFO_COLUMN in s.columns:
        val = _to_float(s.iloc[0][INFO_COLUMN])
        units = str(s.iloc[0]["Units"]).lower() if "Units" in s.columns else ""
        bps = _rate_to_bps(val, units) if val is not None else None
        if bps and bps > 0:
            return bps
    # Parameter patterns
    pat = re.compile("|".join([
        r"^Supported\s+user\s+data\s+rate",
        r"Uplink\s+data\s+rate",
        r"Telecommand\s+data\s+rate",
        r"Command\s+data\s+rate",
        r"\bTC\s*rate\b",
    ]), re.I)
    s2 = df[df["Parameter"].astype(str).str.contains(pat, na=False)]
    if not s2.empty and INFO_COLUMN in s2.columns:
        val = _to_float(s2.iloc[0][INFO_COLUMN])
        units = str(s2.iloc[0]["Units"]).lower() if "Units" in s2.columns else ""
        bps = _rate_to_bps(val, units) if val is not None else None
        if bps and bps > 0:
            return bps
    return None

# ---------------- geometry ----------------
def central_angle_from_slant(range_km: float, alt_km: float) -> float:
    H = R_EARTH + alt_km
    rho = range_km
    cos_theta = (R_EARTH*R_EARTH + H*H - rho*rho) / (2*R_EARTH*H)
    return math.acos(max(-1.0, min(1.0, cos_theta)))

def mean_motion_rad_s(alt_km: float) -> float:
    a = R_EARTH + alt_km
    return math.sqrt(MU_EARTH / (a*a*a))

# ---------------- core budget ----------------
def build_budget(link_csv: str, alt_km: float,
                 emin: float | None, margin_min: float,
                 rate_bps: float) -> pd.DataFrame:
    if rate_bps <= 0:
        raise ValueError("rate_bps must be > 0")

    df = pd.read_csv(link_csv)
    need = {"orbit_altitude_km","elevation_deg","link_margin_db","slant_range_km"}
    if not need.issubset(df.columns):
        raise ValueError(f"{Path(link_csv).name} missing columns: {need - set(df.columns)}")

    dfa = df[df["orbit_altitude_km"].astype(float) == float(alt_km)].copy()
    if dfa.empty:
        raise ValueError(f"No rows for orbit_altitude_km={alt_km} in {Path(link_csv).name}")
    dfa.sort_values("elevation_deg", inplace=True)

    if emin is None:
        emin = float(dfa["elevation_deg"].min())

    e = dfa["elevation_deg"].to_list()
    m = dfa["link_margin_db"].to_list()
    r = dfa["slant_range_km"].to_list()
    theta = [central_angle_from_slant(x, alt_km) for x in r]
    n = mean_motion_rad_s(alt_km)

    rows = []
    for i in range(len(e)-1):
        e0, e1 = e[i], e[i+1]
        m0, m1 = m[i], m[i+1]
        th0, th1 = theta[i], theta[i+1]

        if max(e0, e1) < emin:
            continue
        e_start, e_end = max(min(e0, e1), emin), max(e0, e1)
        if e_end <= e_start:
            continue

        def lerp(x0, y0, x1, y1, x):
            t = (x - x0) / (x1 - x0)
            return y0 + t * (y1 - y0)

        # ensure ascending for interpolation
        if e1 < e0:
            e0, e1 = e1, e0
            m0, m1 = m1, m0
            th0, th1 = th1, th0

        th_start = lerp(e0, th0, e1, th1, e_start)
        th_end   = lerp(e0, th0, e1, th1, e_end)
        m_start  = lerp(e0, m0,  e1, m1,  e_start)
        m_end    = lerp(e0, m0,  e1, m1,  e_end)

        # time across BOTH sides of pass (rise + set)
        dtheta = max(0.0, th_start - th_end)
        bin_time_s = 2.0 * dtheta / n

        # fraction of this bin that meets the margin threshold
        if (m_start < margin_min) and (m_end < margin_min):
            usable_fraction = 0.0
        elif (m_start >= margin_min) and (m_end >= margin_min):
            usable_fraction = 1.0
        else:
            if m_end != m_start:
                e_cross = e_start + (margin_min - m_start) * (e_end - e_start) / (m_end - m_start)
                usable_fraction = (e_end - e_cross) / (e_end - e_start) if e_start <= e_cross <= e_end else (1.0 if m_end >= margin_min else 0.0)
            else:
                usable_fraction = 1.0 if m_start >= margin_min else 0.0

        data_bits = rate_bps * bin_time_s * usable_fraction
        data_MB   = data_bits / (8 * 1e6)   # MB (decimal)
        data_MiB  = data_bits / (8 * 1024**2)

        rows.append({
            "alt_km": float(alt_km),
            "elev_start_deg": float(e_start),
            "elev_end_deg": float(e_end),
            "theta_start_deg": math.degrees(th_start),
            "theta_end_deg": math.degrees(th_end),
            "bin_time_s": bin_time_s,
            "margin_start_db": m_start,
            "margin_end_db": m_end,
            "usable_fraction": usable_fraction,
            "rate_bps": rate_bps,
            "data_bits": data_bits,
            "data_MB": data_MB,
            "data_MiB": data_MiB,
        })

    return pd.DataFrame(rows)

# ---------------- summary & plotting ----------------
def summarise_and_plot(all_results: list[pd.DataFrame], outdir: Path):
    if not all_results:
        return
    big = pd.concat(all_results, ignore_index=True)
    # totals by alt x rate
    grp = big.groupby(["alt_km","rate_bps"], as_index=False).agg(
        total_time_s = ("bin_time_s","sum"),
        total_MB     = ("data_MB","sum"),
    )
    grp["rate_kbps"] = grp["rate_bps"] / 1e3
    # save summary CSV
    out_csv = outdir / "uplink_pass_capacity_summary.csv"
    grp.sort_values(["alt_km","rate_bps"]).to_csv(out_csv, index=False)
    print(f"Wrote summary → {out_csv}")

    # grouped bar chart: x=rates, bars by altitude
    pivot = grp.pivot(index="rate_kbps", columns="alt_km", values="total_MB").sort_index()
    fig, ax = plt.subplots(figsize=(9,6))
    alts = list(pivot.columns)
    x = np.arange(len(pivot.index))
    group_width = 0.8
    bar_w = group_width / max(1, len(alts))
    for i, alt in enumerate(alts):
        offset = (i - (len(alts)-1)/2) * bar_w
        bars = ax.bar(x + offset, pivot[alt].values, width=bar_w, label=f"{int(alt)} km")
        try:
            ax.bar_label(bars, padding=2, fmt="%.2f")
        except Exception:
            pass
    ax.set_title("S-band Uplink Capacity per Pass (MB) vs Rate")
    ax.set_xlabel("Uplink rate (kbps)")
    ax.set_ylabel("MB per pass (≥ margin threshold)")
    ax.set_xticks(x)
    ax.set_xticklabels([f"{int(r)}" for r in pivot.index], rotation=0)
    ax.grid(True, axis="y")
    ax.legend(title="Altitude")
    fig.tight_layout()
    out_png = outdir / "uplink_pass_capacity_summary.png"
    fig.savefig(out_png, bbox_inches="tight")
    plt.close(fig)
    print(f"Wrote chart  → {out_png}")

# ---------------- CLI / Auto ----------------
def main():
    ap = argparse.ArgumentParser(description="S-band UPLINK pass data budget — Satlab SRS-4")
    ap.add_argument("--link-csv", help="Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv")
    ap.add_argument("--info-csv", help="s_up_info.csv (only needed if --use-info-rate)")
    ap.add_argument("--use-info-rate", action="store_true", help="Include a single rate parsed from s_up_info.csv")
    ap.add_argument("--rate-bps", type=float, action="append",
                    help="Uplink rate in bits/s. Repeat to add multiple rates.")
    ap.add_argument("--alt-km", type=float, help="Single altitude to compute")
    ap.add_argument("--emin", type=float, default=None, help="Minimum elevation (deg) for integration")
    ap.add_argument("--margin-min", type=float, default=0.0, help="Minimum link margin counted (dB)")
    ap.add_argument("--outdir", default=".", help="Output directory")
    args, _ = ap.parse_known_args()

    # Resolve files (auto mode if none provided)
    link_csv = args.link_csv or find_file("Satlab_SRS-4_S-band_Uplink_link_margin_vs_elevation.csv")
    info_csv = args.info_csv or find_file("s_up_info.csv")
    if not link_csv:
        raise SystemExit("Could not find the uplink link-curve CSV. Pass --link-csv.")

    outdir = Path(args.outdir); outdir.mkdir(parents=True, exist_ok=True)

    # Build rate set
    rates = []
    if args.rate_bps:
        rates.extend([r for r in args.rate_bps if r and r > 0])
    if args.use_info_rate:
        ir = get_info_rate_bps(info_csv) if info_csv else None
        if ir and ir > 0:
            rates.append(ir)
        else:
            print("NOTE: --use-info-rate given but no valid rate found in s_up_info.csv.")
    if not rates:
        rates = DEFAULT_RATES_BPS[:]  # use defaults

    # Altitudes to process
    if args.alt_km is not None:
        alts = [float(args.alt_km)]
    else:
        dft = pd.read_csv(link_csv)
        if "orbit_altitude_km" not in dft.columns:
            raise SystemExit("Link CSV missing 'orbit_altitude_km' column.")
        alts = sorted(set(map(float, dft["orbit_altitude_km"].unique())))

    # Compute per alt x rate, write per-file, collect for summary
    all_results = []
    for alt in alts:
        for r_bps in rates:
            df = build_budget(link_csv, alt, args.emin, args.margin_min, r_bps)
            rkb = int(round(r_bps/1e3))
            out_csv = outdir / f"pass_data_budget_uplink_{int(alt)}km_{rkb}kbps.csv"
            df.to_csv(out_csv, index=False)
            all_results.append(df)
            print(f"[{int(alt)} km @ {rkb} kbps] time≥thr: {df['bin_time_s'].sum():.1f}s  data: {df['data_MB'].sum():.3f} MB → {out_csv.name}")

    summarise_and_plot(all_results, outdir)

if __name__ == "__main__":
    main()


[500 km @ 10 kbps] time≥thr: 396.9s  data: 0.417 MB → pass_data_budget_uplink_500km_10kbps.csv
[500 km @ 19 kbps] time≥thr: 396.9s  data: 0.835 MB → pass_data_budget_uplink_500km_19kbps.csv
[500 km @ 64 kbps] time≥thr: 396.9s  data: 2.783 MB → pass_data_budget_uplink_500km_64kbps.csv
[500 km @ 100 kbps] time≥thr: 396.9s  data: 4.349 MB → pass_data_budget_uplink_500km_100kbps.csv
[500 km @ 256 kbps] time≥thr: 396.9s  data: 11.133 MB → pass_data_budget_uplink_500km_256kbps.csv
[600 km @ 10 kbps] time≥thr: 422.5s  data: 0.355 MB → pass_data_budget_uplink_600km_10kbps.csv
[600 km @ 19 kbps] time≥thr: 422.5s  data: 0.711 MB → pass_data_budget_uplink_600km_19kbps.csv
[600 km @ 64 kbps] time≥thr: 422.5s  data: 2.370 MB → pass_data_budget_uplink_600km_64kbps.csv
[600 km @ 100 kbps] time≥thr: 422.5s  data: 3.703 MB → pass_data_budget_uplink_600km_100kbps.csv
[600 km @ 256 kbps] time≥thr: 422.5s  data: 9.479 MB → pass_data_budget_uplink_600km_256kbps.csv
Wrote summary → uplink_pass_capacity_summ